This is data mining code for the HAP780 final project after exporting the datasets from All of Us

In [ ]:
# Import library
import pandas as pd

# Load the datasets
df_analysis = pd.read_csv('./data/df_analysis_final.csv')

In [ ]:
# import libraries
import numpy as np
from scipy.stats import chi2_contingency # for chi-square tests

In [ ]:
# import library
from scipy.stats import ttest_ind

Feature Creation: Age as dummy variables in decades

In [ ]:
# Define bins
bins = [0, 18, 28, 38, 48, 58, 68, 78, 88, 98, float('inf')]
labels = ['<18', '18-27', '28-37', '38-47', '48-57', '58-67', '68-77', '78-87', '88-97', '98+']

# Cut the age_at_first_diagnosis into bins
df_analysis['age_group'] = pd.cut(df_analysis['age_at_first_diagnosis'], bins=bins, labels=labels, right=False)

# Convert the binned data into dummy variables
age_dummies = pd.get_dummies(df_analysis['age_group'])

# Concatenate the dummy variables with the original dataframe if needed
df_analysis = pd.concat([df_analysis, age_dummies], axis=1)

# Drop the 'age_at_first_diagnosis' and 'age_group' columns from the dataframe
df_analysis = df_analysis.drop(['age_at_first_diagnosis', 'age_group'], axis=1)

In [ ]:
df_analysis.head()

Feature Creation: Create Interaction Variables (2-way only due to memory constraints)

In [ ]:
# Import library
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# Drop the target variable
X = df_analysis.drop(columns=['No_remission'])

# Create polynomial features (interaction terms only)
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_poly = poly.fit_transform(X)

# Create a dataframe for the interaction terms
df_interactions = pd.DataFrame(X_poly, columns=poly.get_feature_names_out(X.columns))

# Identify interaction terms only (exclude original feature columns)
interaction_columns = df_interactions.columns[~df_interactions.columns.isin(X.columns)]

# Join the original df_analysis with the interaction terms dataframe
df_analysis_extended = pd.concat([df_analysis, df_interactions[interaction_columns]], axis=1)

Data cleaning: Remove columns that have all zeroes or all ones

In [ ]:
# Get all columns that are all ones or all zeroes for dropping
columns_to_drop = df_analysis_extended.columns[(
    df_analysis_extended.sum(axis=0) == len(df_analysis_extended)) | (df_analysis_extended.sum(axis=0) == 0)]

# Drop these columns
df_analysis_extended = df_analysis_extended.drop(columns=columns_to_drop)

In [ ]:
df_analysis_extended.head()

In [ ]:
df_analysis_extended.info()

Split the data into Training (80%) and Test (20%)

In [ ]:
# Import library
from sklearn.model_selection import train_test_split

In [ ]:
# Splitting the data into training and test sets
train_set, test_set = train_test_split(df_analysis_extended, test_size=0.20, random_state=42)

In [ ]:
train_set.head()

In [ ]:
train_set.info()

In [ ]:
test_set.head()

In [ ]:
test_set.info()

Baseline models without feature selection

In [ ]:
# Import libraries
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import (confusion_matrix, 
                             precision_score, 
                             recall_score, 
                             f1_score, 
                             matthews_corrcoef, 
                             roc_auc_score, 
                             average_precision_score)

In [ ]:
# Splitting the data into features and target
X_train = train_set.drop(columns=['No_remission'])
y_train = train_set['No_remission']


# Splitting the test data into features and target
X_test = test_set.drop(columns=['No_remission'])
y_test = test_set['No_remission']

Feature Selection: LASSO Regression with Cross-Validation

In [ ]:
# Import library
from sklearn.linear_model import LassoCV

In [ ]:
# Initializing LassoCV (5 folds)
lasso_cv = LassoCV(cv=5, random_state=42)

# Fitting the model
lasso_cv.fit(X_train, y_train)

# Get the feature coefficients
coef = pd.Series(lasso_cv.coef_, index=X_train.columns)

# Filter out the features which have a coefficient of zero
selected_features_lasso = coef[coef != 0].index.tolist()

In [ ]:
# Print selected feature and direction
print(f"Number of features selected: {len(selected_features_lasso)}\n")

for feature, coef in zip(selected_features_lasso, lasso_cv.coef_):
    effect = 'increase' if coef > 0 else 'decrease' if coef < 0 else 'no effect'
    print(f"feature: '{feature}' | effect: {effect}")

In [ ]:
# Filter train and test sets for selected features
X_train_selected_lasso = X_train[selected_features_lasso]
X_test_selected_lasso = X_test[selected_features_lasso]

Class Balancing Using Synthetic Minority Over-sampling Technique (SMOTE)

In [ ]:
# Import libraries
from imblearn.over_sampling import SMOTE

In [ ]:
# Using SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Resampled sample size
print(f"Original training sample size: {X_train.shape[0]}")
print(f"Resampled training sample size: {X_resampled.shape[0]}")

Model Training and Testing Using Balanced Data and Selected Features (LASSO)

In [ ]:
# Filter train and test sets for selected features
X_train_selected_lasso_2 = X_resampled[selected_features_lasso]
X_test_selected_lasso_2 = X_test[selected_features_lasso]

# Define models
models = {
    'Logistic Regression': LogisticRegression(max_iter=5000),
    'Random Forest': RandomForestClassifier(),
    'Naive Bayes': GaussianNB(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric="logloss")
}

# Metrics collection
results = {}

for name, model in models.items():
    # Train model
    model.fit(X_train_selected_lasso_2, y_resampled)
    
    # Predict
    y_pred = model.predict(X_test_selected_lasso_2)
    
    # Metrics
    confusion = confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    fmeasure = f1_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    roc_area = roc_auc_score(y_test, model.predict_proba(X_test_selected_lasso_2)[:, 1])
    prc_area = average_precision_score(y_test, model.predict_proba(X_test_selected_lasso_2)[:, 1])
    
    results[name] = {
        'Confusion Matrix': confusion,
        'Precision': precision,
        'Recall': recall,
        'F-Measure': fmeasure,
        'MCC': mcc,
        'ROC Area': roc_area,
        'PRC Area': prc_area
    }
    
# Display results
for name, metrics in results.items():
    print(f"Model: {name}")
    for metric_name, metric_value in metrics.items():
        print(f"{metric_name}: {metric_value}")
    print("\n")

Hyperparameter Tuning Of Best Data and Feature Combination On Recall
- Scoring will be based on recall
- Chosen training set with highest recall: Balanced with Feature Selection 

In [ ]:
# Import library
from sklearn.model_selection import GridSearchCV

In [ ]:
# Logistic Regression

# Define training sets as balanced with feature selection (X_train_selected_lasso_2, y_resampled)
# Define test set as balanced with feature selection (X_test_selected_lasso_2)

# Define the parameter grid for Logistic Regression
param_grid_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

# Initialize the GridSearchCV object for Logistic Regression
grid_search_lr = GridSearchCV(estimator=LogisticRegression(solver='liblinear'), 
                              param_grid=param_grid_lr, 
                              scoring=['recall'], 
                              refit='recall', 
                              cv=5)

# Fit the grid search to the data
grid_search_lr.fit(X_train_selected_lasso_2, y_resampled)

# After fitting, we can check the best performance in the training set
print("Best parameters set found on training set:")
print(grid_search_lr.best_params_)

# Predict
best_estimator = grid_search_lr.best_estimator_
y_pred = best_estimator.predict(X_test_selected_lasso_2)
    
# Metrics
confusion = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
fmeasure = f1_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
roc_area = roc_auc_score(y_test, best_estimator.predict_proba(X_test_selected_lasso_2)[:, 1])
prc_area = average_precision_score(y_test, best_estimator.predict_proba(X_test_selected_lasso_2)[:, 1])
    
results = {
    'Confusion Matrix': confusion,
    'Precision': precision,
    'Recall': recall,
    'F-Measure': fmeasure,
    'MCC': mcc,
    'ROC Area': roc_area,
    'PRC Area': prc_area
}
    
# Display results
for metric_name, metric_value in results.items():
    print(f"{metric_name}: {metric_value}")

# Save best estimator for plotting
best_logreg = best_estimator

In [ ]:
# Random Forest

# Define training sets as balanced with feature selection (X_train_selected_lasso_2, y_resampled)
# Define test set as balanced with feature selection (X_test_selected_lasso_2)

# Define the parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the GridSearchCV object for Random Forest
grid_search_rf = GridSearchCV(estimator=RandomForestClassifier(), 
                              param_grid=param_grid_rf, 
                              scoring=['recall'], 
                              refit='recall', 
                              cv=5)

# Fit the grid search to the data
grid_search_rf.fit(X_train_selected_lasso_2, y_resampled)

# After fitting, we can check the best performance in the training set
print("Best parameters set found on training set:")
print(grid_search_rf.best_params_)

# Predict
best_estimator = grid_search_rf.best_estimator_
y_pred = best_estimator.predict(X_test_selected_lasso_2)
    
# Metrics
confusion = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
fmeasure = f1_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
roc_area = roc_auc_score(y_test, best_estimator.predict_proba(X_test_selected_lasso_2)[:, 1])
prc_area = average_precision_score(y_test, best_estimator.predict_proba(X_test_selected_lasso_2)[:, 1])
    
results = {
    'Confusion Matrix': confusion,
    'Precision': precision,
    'Recall': recall,
    'F-Measure': fmeasure,
    'MCC': mcc,
    'ROC Area': roc_area,
    'PRC Area': prc_area
}
    
# Display results
for metric_name, metric_value in results.items():
    print(f"{metric_name}: {metric_value}")

# Save best estimator for plotting
best_rf = best_estimator

In [ ]:
# Naive Bayes

# Define training sets as balanced with feature selection (X_train_selected_lasso_2, y_resampled)
# Define test set as balanced with feature selection (X_test_selected_lasso_2)

# Define the parameter grid for GaussianNB
param_grid_gnb = {
    'var_smoothing': np.logspace(0,-9, num=100)
}

# Initialize the GridSearchCV object for GaussianNB
grid_search_gnb = GridSearchCV(estimator=GaussianNB(), 
                               param_grid=param_grid_gnb, 
                               scoring=['recall'], 
                               refit='recall', 
                               cv=5)

# Fit the grid search to the data
grid_search_gnb.fit(X_train_selected_lasso_2, y_resampled)

# After fitting, we can check the best performance in the training set
print("Best parameters set found on training set:")
print(grid_search_gnb.best_params_)

# Predict
best_estimator = grid_search_gnb.best_estimator_
y_pred = best_estimator.predict(X_test_selected_lasso_2)
    
# Metrics
confusion = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
fmeasure = f1_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
roc_area = roc_auc_score(y_test, best_estimator.predict_proba(X_test_selected_lasso_2)[:, 1])
prc_area = average_precision_score(y_test, best_estimator.predict_proba(X_test_selected_lasso_2)[:, 1])
    
results = {
    'Confusion Matrix': confusion,
    'Precision': precision,
    'Recall': recall,
    'F-Measure': fmeasure,
    'MCC': mcc,
    'ROC Area': roc_area,
    'PRC Area': prc_area
}
    
# Display results
for metric_name, metric_value in results.items():
    print(f"{metric_name}: {metric_value}")

# Save best estimator for plotting
best_nb = best_estimator

In [ ]:
# XGBoost

# Define training sets as balanced with feature selection (X_train_selected_lasso_2, y_resampled)
# Define test set as balanced with feature selection (X_test_selected_lasso_2)

# Define the parameter grid for XGBoost
param_grid_xgb = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7],
    'colsample_bytree': [0.3, 0.7, 1]
}

# Initialize the GridSearchCV object for XGBoost
grid_search_xgb = GridSearchCV(estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
                               param_grid=param_grid_xgb, 
                               scoring=['recall'], 
                               refit='recall', 
                               cv=5)

# Fit the grid search to the data
grid_search_xgb.fit(X_train_selected_lasso_2, y_resampled)

# After fitting, we can check the best performance in the training set
print("Best parameters set found on training set:")
print(grid_search_xgb.best_params_)

# Predict
best_estimator = grid_search_xgb.best_estimator_
y_pred = best_estimator.predict(X_test_selected_lasso_2)
    
# Metrics
confusion = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
fmeasure = f1_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
roc_area = roc_auc_score(y_test, best_estimator.predict_proba(X_test_selected_lasso_2)[:, 1])
prc_area = average_precision_score(y_test, best_estimator.predict_proba(X_test_selected_lasso_2)[:, 1])
    
results = {
    'Confusion Matrix': confusion,
    'Precision': precision,
    'Recall': recall,
    'F-Measure': fmeasure,
    'MCC': mcc,
    'ROC Area': roc_area,
    'PRC Area': prc_area
}
    
# Display results
for metric_name, metric_value in results.items():
    print(f"{metric_name}: {metric_value}")

# Save best estimator for plotting
best_xgb = best_estimator

Plotting the ROC curves for the hyperparameter tuned models

In [ ]:
# Import the libraries
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [ ]:
# Replace with your actual models and test data
models = {
    'Logistic Regression': best_logreg,
    'Random Forest': best_rf,
    'Naive Bayes': best_nb,
    'XGBoost': best_xgb
}

plt.figure(figsize=(10, 8))

# Calculate ROC curve and ROC AUC for each model
for name, model in models.items():
    probas_ = model.predict_proba(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, probas_[:, 1])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{name} (area = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()
